"""
RESUMEN DE TRANSFORMACIONES APLICADAS:

1. LEADS:
   - Normalización de nombres de columnas
   - Validación de teléfonos (formato español)
   - Procesamiento robusto de edad con tipo Int64
   - Conversión de formato europeo de números
   - Validación de escalas (urgencia 1-5, lead_score 0-100)
   - Creación de categorías demográficas y de negocio

2. CURSOS:
   - Eliminación de registros inválidos
   - Normalización de texto y espacios
   - Conversión de tipos numéricos
   - Formateo de fechas
   - Cálculo de métricas derivadas (descuento, intensidad)
   - Categorización por precio

3. SEGUIMIENTO:
   - Eliminación de columnas irrelevantes
   - Conversión de fechas y booleanos
   - Limpieza robusta de columnas numéricas
   - Normalización de categorías (Title Case)
   - Eliminación de valores negativos
   - Deduplicación por ID

4. PRIMER CONTACTO:
   - Eliminación de columnas irrelevantes
   - Limpieza completa de texto
   - Procesamiento de fechas y horas (formato SQL)
   - Imputación inteligente de valores faltantes
   - Conversión de tipos optimizados
   - Deduplicación completa

ARCHIVOS GENERADOS:
- leads_limpio.csv
- cursos_limpio.csv  
- seguimiento_limpio.csv
- primer_contacto_limpio.csv

Los datos están ahora listos para análisis exploratorio, visualización
y modelado predictivo.
"""

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime, timedelta

# Analizamos los datos del fichero 'Leads'

In [2]:
df_leads = pd.read_csv('../data/Leads.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')


In [3]:
df_leads

,lead_id,curso_interes_id,curso_interes_nombre,fecha_registro,fuente,campana,nombre,apellidos,email,telefono,...,presupuesto_estimado,urgencia_formacion,dispositivo_registro,horario_preferido,modalidad_preferida,comercial_asignado,lead_score,Codigo,ComprobaciónAleatoria,ComprobaciónAleatoria2
0,5b20df2f-49d1-49f2-bff4-e68e112ff6b4,87c56ad1,Data Science,2025-03-04,Facebook Ads,FB_Promocion_Mayo,Feliciana,Cantón Amaya,feliciana.canton@hotmail.com,627680402.0,...,5741,2.0,Móvil,Noche,Híbrido,Ana García,45.0,3172,VERDADERO,VERDADERO
1,e40d99b0-a58c-4a6d-9779-d4e9d7f1f83a,87c56ad1,Data Science,2024-09-26,Google Ads,GG_Search_Cursos,Leonardo,Hierro Godoy,leonardo.hierro@gmail.com,693008687.0,...,5958,3.0,Móvil,Mañana,Online,Carlos Martínez,58.0,3251,VERDADERO,VERDADERO
2,e6f9dc7e-6c56-4a4e-b8bb-c61f2f018301,c0e24ebc,Data Analytics,2025-01-01,YouTube,YT_Testimonial,Maite,Calatayud Bonet,maite.calatayud@hotmail.com,748136760.0,...,2814,5.0,Móvil,Noche,Híbrido,Sofía Fernández,53.0,5619,FALSO,FALSO
3,8808cd20-a01b-41b1-8cbb-22fe18f1314b,6fa49962,Desarrollo Web Full Stack,2025-04-14,Email Marketing,Email_Promo,Telmo,Antón Andrés,telmo.anton@gmail.com,709572486.0,...,5255,4.0,Ordenador,Fin de semana,Online,Carlos Martínez,77.0,9276,VERDADERO,VERDADERO
4,f17f1eb6-babe-44c4-a08c-2be3d6498319,6ae1ccdb,Ciberseguridad,2025-03-15,Referido,Referral_Alumnos,Isaías,Guerrero Vazquez,isaias.guerrero@hotmail.com,667403959.0,...,3225,3.0,Móvil,Fin de semana,Presencial,Ana García,66.0,6749,VERDADERO,FALSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105240,e07d5dce-456a-4718-ba3a-ffa1ccd64e55,87c56ad1,Data Science,2024-11-06,FACEBOOK ADS,FB_Career_2025,LUZ,Calzada Camacho,luz.calzada@gmail.com,NaN,...,NaN,2.0,Móvil,Tarde,PRESENCIAL,CARLOS MARTÍNEZ,22.195.820.936.509.700,5579,VERDADERO,VERDADERO
105241,A42FDCBF-E951-4D38-B860-576F5FB7BA45,87C56AD1,Data Science,2025-01-12,TIKTOK,TT_Challenge,Laura,Barberá Almagro,laura.barbera@gmail.com,NaN,...,"12645,856280713935",3.0,Móvil,Tarde,Presencial,ANA GARCÍA,2.035.637.284.296.640,6270,FALSO,FALSO
105242,Dc5a9362-b4b8-40f3-a7f3-6f91996de995,181bd83f,IA DEVELOPER,2024-10-24,Google ads,Gg_remarketing,RAFAEL,ESPINOSA FUERTES,Rafael.espinosa@hotmail.com,20.886.693.070.998.900,...,NaN,4.0,MÓVIL,Noche,Híbrido,Ana García,15.968.613.850.495.000,2068,VERDADERO,FALSO
105243,Aaea5f50-707f-43b4-9278-eea79ff5b5f9,6AE1CCDB,CIBERSEGURIDAD,2024-08-04,Google Ads,Gg_remarketing,Adelia,POZUELO BLÁZQUEZ,adelia.pozuelo@gmail.com,6.352.937.068.343.480,...,"2731,176752517995",2.0,ORDENADOR,FLEXIBLE,Presencial,MIGUEL GONZÁLEZ,NaN,4358,VERDADERO,FALSO


In [4]:
df_leads.shape

(105245, 26)

In [5]:
df_leads.columns

Index(['lead_id', 'curso_interes_id', 'curso_interes_nombre', 'fecha_registro',
       'fuente', 'campana', 'nombre', 'apellidos', 'email', 'telefono', 'edad',
       'ciudad', 'provincia', 'nivel_estudios', 'situacion_laboral',
       'objetivo_profesional', 'presupuesto_estimado', 'urgencia_formacion',
       'dispositivo_registro', 'horario_preferido', 'modalidad_preferida',
       'comercial_asignado', 'lead_score', 'Codigo', 'ComprobaciónAleatoria',
       'ComprobaciónAleatoria2'],
      dtype='object')

In [6]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105245 entries, 0 to 105244
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   lead_id                 105245 non-null  object
 1   curso_interes_id        105245 non-null  object
 2   curso_interes_nombre    105245 non-null  object
 3   fecha_registro          105245 non-null  object
 4   fuente                  105245 non-null  object
 5   campana                 105245 non-null  object
 6   nombre                  105245 non-null  object
 7   apellidos               105245 non-null  object
 8   email                   105245 non-null  object
 9   telefono                103768 non-null  object
 10  edad                    103088 non-null  object
 11  ciudad                  105245 non-null  object
 12  provincia               105245 non-null  object
 13  nivel_estudios          105245 non-null  object
 14  situacion_laboral       105245 non-n

In [7]:
df_leads.columns = df_leads.columns.str.lower()

In [8]:
df_leads.columns

Index(['lead_id', 'curso_interes_id', 'curso_interes_nombre', 'fecha_registro',
       'fuente', 'campana', 'nombre', 'apellidos', 'email', 'telefono', 'edad',
       'ciudad', 'provincia', 'nivel_estudios', 'situacion_laboral',
       'objetivo_profesional', 'presupuesto_estimado', 'urgencia_formacion',
       'dispositivo_registro', 'horario_preferido', 'modalidad_preferida',
       'comercial_asignado', 'lead_score', 'codigo', 'comprobaciónaleatoria',
       'comprobaciónaleatoria2'],
      dtype='object')

In [9]:
if 'fecha_registro' in df_leads.columns:
    df_leads['fecha_registro'] = pd.to_datetime(df_leads['fecha_registro'])

In [10]:
df_leads['telefono'] = df_leads['telefono'].str.replace('.0', '', regex=False)

In [11]:
df_leads.loc[df_leads['telefono'].str.len() > 9, 'telefono'] = None

In [12]:
if 'edad' in df_leads.columns:
    df_leads['edad'] = pd.to_numeric(df_leads["edad"], errors= "coerce")
    df_leads['edad'] = df_leads['edad'].fillna(0).astype(int)
    df_leads['edad'] = df_leads['edad'].replace(0, np.nan)
    df_leads["edad"] = df_leads["edad"].astype('Int64')
    

In [13]:
df_leads['presupuesto_estimado']  

0                       5741
1                       5958
2                       2814
3                       5255
4                       3225
                 ...        
105240                   NaN
105241    12645,856280713935
105242                   NaN
105243     2731,176752517995
105244                   NaN
Name: presupuesto_estimado, Length: 105245, dtype: object

In [14]:
df_leads ['presupuesto_estimado'] = df_leads ['presupuesto_estimado'].str.replace('.', '').str.replace(',', '.').astype(float)


In [15]:
df_leads ['presupuesto_estimado']

0          5741.000000
1          5958.000000
2          2814.000000
3          5255.000000
4          3225.000000
              ...     
105240             NaN
105241    12645.856281
105242             NaN
105243     2731.176753
105244             NaN
Name: presupuesto_estimado, Length: 105245, dtype: float64

In [16]:
df_leads['presupuesto_estimado'] = df_leads['presupuesto_estimado'].fillna(0)
df_leads['presupuesto_estimado'] = df_leads['presupuesto_estimado'].round(2).astype(float)


In [17]:
df_leads ['presupuesto_estimado']

0          5741.00
1          5958.00
2          2814.00
3          5255.00
4          3225.00
            ...   
105240        0.00
105241    12645.86
105242        0.00
105243     2731.18
105244        0.00
Name: presupuesto_estimado, Length: 105245, dtype: float64

In [18]:
if 'urgencia_formacion' in df_leads.columns:
    df_leads['urgencia_formacion'] = pd.to_numeric(df_leads['urgencia_formacion'], errors='coerce')
    df_leads['urgencia_formacion'] = df_leads['urgencia_formacion'].apply(lambda x: np.nan if pd.notna(x) and x > 5 else x)
    df_leads['urgencia_formacion'] = df_leads['urgencia_formacion'].astype('Int64')
    

In [19]:
if 'lead_score' in df_leads.columns:
    df_leads['lead_score'] = pd.to_numeric(df_leads['lead_score'], errors='coerce')
    df_leads['lead_score'] = df_leads['lead_score'].apply(lambda x: np.nan if pd.notna(x) and x > 100 else x)
    df_leads['lead_score'] = df_leads['lead_score'].astype('Int64')

In [20]:
if 'codigo' in df_leads.columns:
    df_leads['codigo'] = df_leads['codigo'].astype(int)

In [21]:
text_columns = ['nombre', 'apellidos', 'email', 'ciudad', 'provincia', 'nivel_estudios', 'situacion_laboral', 'objetivo_profesional','dispositivo_registro', 'horario_preferido', 'modalidad_preferida',
'comercial_asignado', 'fuente', 'campana']

In [22]:
for col in text_columns:
    if col in df_leads.columns:
        df_leads[col] = df_leads[col].str.strip()
        if col in ['nombre', 'apellidos', 'ciudad', 'provincia', 'comercial_asignado']:
            df_leads[col] = df_leads[col].str.title()

In [23]:
df_leads['email'] = df_leads['email'].str.lower()


In [24]:
if 'lead_id' in df_leads.columns:
    duplicados_id = df_leads['lead_id'].duplicated().sum()

In [25]:
if 'email' in df_leads.columns:
    duplicados_email = df_leads['email'].duplicated().sum()
    print(f"Duplicados por email: {duplicados_email}")
    if duplicados_email > 0:
        print("Emails duplicados:")
        print(df_leads[df_leads['email'].duplicated(keep=False)]['email'].value_counts())

Duplicados por email: 4869
Emails duplicados:
email
oriana.teruel@hotmail.com        4
micaela.armengol@hotmail.com     4
reyes.rios@gmail.com             3
victor.munoz@yahoo.com           3
eloisa.rius@hotmail.com          3
                                ..
carlota.landa@gmail.com          2
nicodemo.marino@gmail.com        2
severiano.cordero@hotmail.com    2
ramiro.barragan@hotmail.com      2
vicente.guzman@hotmail.com       2
Name: count, Length: 4742, dtype: int64


In [26]:
duplicados_totales = df_leads.duplicated().sum()
if duplicados_totales > 0:
    df_leads = df_leads.drop_duplicates()

In [27]:
if 'edad' in df_leads.columns:
    df_leads['grupo_edad'] = pd.cut(df_leads['edad'], 
                             bins=[0, 25, 35, 45, 55, 100], 
                             labels=['18-25', '26-35', '36-45', '46-55', '56+'],
                             right=False)

In [28]:
if 'presupuesto_estimado' in df_leads.columns:
    df_leads['categoria_presupuesto'] = pd.cut(df_leads['presupuesto_estimado'], 
                                        bins=[0, 3000, 5000, 7000, 100000], 
                                        labels=['Bajo', 'Medio', 'Alto', 'Pro'],
                                        right=False)

In [29]:
if 'lead_score' in df_leads.columns:
    df_leads['calidad_lead'] = pd.cut(df_leads['lead_score'], 
                               bins=[0, 40, 60, 80, 100], 
                               labels=['Bajo', 'Medio', 'Alto', 'Excelente'],
                               right=False)

In [30]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105245 entries, 0 to 105244
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   lead_id                 105245 non-null  object        
 1   curso_interes_id        105245 non-null  object        
 2   curso_interes_nombre    105245 non-null  object        
 3   fecha_registro          105245 non-null  datetime64[ns]
 4   fuente                  105245 non-null  object        
 5   campana                 105245 non-null  object        
 6   nombre                  105245 non-null  object        
 7   apellidos               105245 non-null  object        
 8   email                   105245 non-null  object        
 9   telefono                99260 non-null   object        
 10  edad                    96513 non-null   Int64         
 11  ciudad                  105245 non-null  object        
 12  provincia               105245

# Analizamos los datos del fichero 'Cursos'

In [31]:
df_cursos = pd.read_csv('../data/Cursos.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')

In [32]:
df_cursos

,Curso ID,Nombre,Modalidad,Duracion meses,Horas totales,Precio base,Precio descuento,Tecnologias,Nivel,Requisitos previos,Fecha inicio proxima,Plazas disponibles,Rating,Empleabilidad,url_dossier,Descripción,Certificación oficial
0,f514fe84,Data Analytics,Híbrido,7.0,364.0,4018.0,3426.0,"Excel, SQL, Tableau, Power BI, Python básico",Avanzado,Conocimientos básicos de ofimática,2025-06-01,22.0,4.2,87.0,https://escuelaformacion.edu/dossier/data-anal...,Conviértete en un profesional del análisis de ...,Sí
1,546ad5a9,Desarrollo Web Full Stack,Presencial,11.0,407.0,6325.0,5701.0,"HTML, CSS, JavaScript, React, Node.js, MongoDB...",Principiante,"Ninguno, se parte desde cero",2025-06-09,16.0,4.8,87.0,https://escuelaformacion.edu/dossier/desarroll...,"Aprende a crear aplicaciones web completas, de...",No
2,f8c5f595,IA Developer,Presencial,4.0,144.0,2180.0,1784.0,"Python, TensorFlow, PyTorch, Scikit-learn, NLP...",Intermedio,"Conocimientos de programación, preferiblemente...",2025-06-07,30.0,4.4,95.0,https://escuelaformacion.edu/dossier/ia-develo...,Especialízate en el desarrollo de soluciones b...,No
3,3530776b,Ciberseguridad,Presencial,5.0,190.0,2175.0,1985.0,"Redes, Pentesting, Análisis forense, Cryptogra...",Avanzado,Conocimientos básicos de redes y sistemas,2025-07-18,23.0,4.7,88.0,https://escuelaformacion.edu/dossier/cibersegu...,Protege sistemas y redes contra vulnerabilidad...,Sí
4,0e567c6f,UX/UI Design,Híbrido,9.0,369.0,4104.0,3284.0,"Figma, Adobe XD, Sketch, Design thinking, User...",Principiante,"Interés por el diseño, no se requiere experien...",2025-07-15,30.0,4.1,76.0,https://escuelaformacion.edu/dossier/ux/ui-des...,Diseña interfaces intuitivas y experiencias de...,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_cursos = df_cursos.dropna(subset=['Curso ID'])
df_cursos = df_cursos.dropna(how='all')

In [34]:
df_cursos

,Curso ID,Nombre,Modalidad,Duracion meses,Horas totales,Precio base,Precio descuento,Tecnologias,Nivel,Requisitos previos,Fecha inicio proxima,Plazas disponibles,Rating,Empleabilidad,url_dossier,Descripción,Certificación oficial
0,f514fe84,Data Analytics,Híbrido,7.0,364.0,4018.0,3426.0,"Excel, SQL, Tableau, Power BI, Python básico",Avanzado,Conocimientos básicos de ofimática,2025-06-01,22.0,4.2,87.0,https://escuelaformacion.edu/dossier/data-anal...,Conviértete en un profesional del análisis de ...,Sí
1,546ad5a9,Desarrollo Web Full Stack,Presencial,11.0,407.0,6325.0,5701.0,"HTML, CSS, JavaScript, React, Node.js, MongoDB...",Principiante,"Ninguno, se parte desde cero",2025-06-09,16.0,4.8,87.0,https://escuelaformacion.edu/dossier/desarroll...,"Aprende a crear aplicaciones web completas, de...",No
2,f8c5f595,IA Developer,Presencial,4.0,144.0,2180.0,1784.0,"Python, TensorFlow, PyTorch, Scikit-learn, NLP...",Intermedio,"Conocimientos de programación, preferiblemente...",2025-06-07,30.0,4.4,95.0,https://escuelaformacion.edu/dossier/ia-develo...,Especialízate en el desarrollo de soluciones b...,No
3,3530776b,Ciberseguridad,Presencial,5.0,190.0,2175.0,1985.0,"Redes, Pentesting, Análisis forense, Cryptogra...",Avanzado,Conocimientos básicos de redes y sistemas,2025-07-18,23.0,4.7,88.0,https://escuelaformacion.edu/dossier/cibersegu...,Protege sistemas y redes contra vulnerabilidad...,Sí
4,0e567c6f,UX/UI Design,Híbrido,9.0,369.0,4104.0,3284.0,"Figma, Adobe XD, Sketch, Design thinking, User...",Principiante,"Interés por el diseño, no se requiere experien...",2025-07-15,30.0,4.1,76.0,https://escuelaformacion.edu/dossier/ux/ui-des...,Diseña interfaces intuitivas y experiencias de...,Sí
5,33a93363,Data Science,Online,5.0,250.0,2200.0,1828.0,"Python, R, SQL, Machine Learning, Deep Learnin...",Intermedio,Conocimientos básicos de programación y estadí...,2025-07-26,17.0,4.4,94.0,https://escuelaformacion.edu/dossier/data-scie...,Domina la ciencia de datos para extraer insigh...,Si
6,e77f35b7,Marketing Digital,Presencial,11.0,418.0,5951.0,4885.0,"SEO, SEM, Social Media, Email Marketing, Analy...",Principiante,"Ninguno, se parte desde cero",2025-05-25,23.0,4.8,85.0,https://escuelaformacion.edu/dossier/marketing...,Desarrolla estrategias de marketing online efe...,Sí


In [35]:
text_columns = ['Nombre', 'Modalidad', 'Tecnologias', 'Nivel', 'Requisitos previos', 'Descripción', 'Certificación oficial']
for col in text_columns:
    if col in df_cursos.columns:
        df_cursos[col] = df_cursos[col].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

In [36]:
df_cursos.shape

(7, 17)

In [37]:
df_cursos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 6
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Curso ID               7 non-null      object 
 1   Nombre                 7 non-null      object 
 2   Modalidad              7 non-null      object 
 3   Duracion meses         7 non-null      float64
 4   Horas totales          7 non-null      float64
 5   Precio base            7 non-null      float64
 6   Precio descuento       7 non-null      float64
 7   Tecnologias            7 non-null      object 
 8   Nivel                  7 non-null      object 
 9   Requisitos previos     7 non-null      object 
 10  Fecha inicio proxima   7 non-null      object 
 11  Plazas disponibles     7 non-null      float64
 12  Rating                 7 non-null      float64
 13  Empleabilidad          7 non-null      float64
 14  url_dossier            7 non-null      object 
 15  Descripción    

In [38]:
numeric_columns = ['Duracion meses', 'Horas totales', 'Precio base', 'Precio descuento', 
                  'Plazas disponibles', 'Rating', 'Empleabilidad']

for col in numeric_columns:
    if col in df_cursos.columns:
        df_cursos[col] = pd.to_numeric(df_cursos[col], errors='coerce')

In [39]:
if 'Fecha inicio proxima' in df_cursos.columns:
        df_cursos['Fecha inicio proxima'] = pd.to_datetime(df_cursos['Fecha inicio proxima'], errors='coerce')
        df_cursos['Fecha inicio proxima'] = df_cursos['Fecha inicio proxima'].dt.strftime('%Y-%m-%d')


In [40]:
if 'Precio base' in df_cursos.columns and 'Precio descuento' in df_cursos.columns:
    df_cursos['Descuento_porcentaje'] = ((df_cursos['Precio base'] - df_cursos['Precio descuento']) / df_cursos['Precio base'] * 100).round(2)

In [41]:
if 'Horas totales' in df_cursos.columns and 'Duracion meses' in df_cursos.columns:
    df_cursos['Horas_por_mes'] = (df_cursos['Horas totales'] / df_cursos['Duracion meses']).round(2)

In [42]:
if 'Precio base' in df_cursos.columns:
    df_cursos['Categoria_precio'] = pd.cut(df_cursos['Precio base'], 
                                         bins=[0, 2000, 5000, float('inf')], 
                                         labels=['Económico', 'Medio', 'Pro'], 
                                         include_lowest=True)

In [43]:
df_cursos = df_cursos.reset_index(drop=True)


In [44]:
df_cursos

,Curso ID,Nombre,Modalidad,Duracion meses,Horas totales,Precio base,Precio descuento,Tecnologias,Nivel,Requisitos previos,Fecha inicio proxima,Plazas disponibles,Rating,Empleabilidad,url_dossier,Descripción,Certificación oficial,Descuento_porcentaje,Horas_por_mes,Categoria_precio
0,f514fe84,Data Analytics,Híbrido,7.0,364.0,4018.0,3426.0,"Excel, SQL, Tableau, Power BI, Python básico",Avanzado,Conocimientos básicos de ofimática,2025-06-01,22.0,4.2,87.0,https://escuelaformacion.edu/dossier/data-anal...,Conviértete en un profesional del análisis de ...,Sí,14.73,52.0,Medio
1,546ad5a9,Desarrollo Web Full Stack,Presencial,11.0,407.0,6325.0,5701.0,"HTML, CSS, JavaScript, React, Node.js, MongoDB...",Principiante,"Ninguno, se parte desde cero",2025-06-09,16.0,4.8,87.0,https://escuelaformacion.edu/dossier/desarroll...,"Aprende a crear aplicaciones web completas, de...",No,9.87,37.0,Pro
2,f8c5f595,IA Developer,Presencial,4.0,144.0,2180.0,1784.0,"Python, TensorFlow, PyTorch, Scikit-learn, NLP...",Intermedio,"Conocimientos de programación, preferiblemente...",2025-06-07,30.0,4.4,95.0,https://escuelaformacion.edu/dossier/ia-develo...,Especialízate en el desarrollo de soluciones b...,No,18.17,36.0,Medio
3,3530776b,Ciberseguridad,Presencial,5.0,190.0,2175.0,1985.0,"Redes, Pentesting, Análisis forense, Cryptogra...",Avanzado,Conocimientos básicos de redes y sistemas,2025-07-18,23.0,4.7,88.0,https://escuelaformacion.edu/dossier/cibersegu...,Protege sistemas y redes contra vulnerabilidad...,Sí,8.74,38.0,Medio
4,0e567c6f,UX/UI Design,Híbrido,9.0,369.0,4104.0,3284.0,"Figma, Adobe XD, Sketch, Design thinking, User...",Principiante,"Interés por el diseño, no se requiere experien...",2025-07-15,30.0,4.1,76.0,https://escuelaformacion.edu/dossier/ux/ui-des...,Diseña interfaces intuitivas y experiencias de...,Sí,19.98,41.0,Medio
5,33a93363,Data Science,Online,5.0,250.0,2200.0,1828.0,"Python, R, SQL, Machine Learning, Deep Learnin...",Intermedio,Conocimientos básicos de programación y estadí...,2025-07-26,17.0,4.4,94.0,https://escuelaformacion.edu/dossier/data-scie...,Domina la ciencia de datos para extraer insigh...,Si,16.91,50.0,Medio
6,e77f35b7,Marketing Digital,Presencial,11.0,418.0,5951.0,4885.0,"SEO, SEM, Social Media, Email Marketing, Analy...",Principiante,"Ninguno, se parte desde cero",2025-05-25,23.0,4.8,85.0,https://escuelaformacion.edu/dossier/marketing...,Desarrolla estrategias de marketing online efe...,Sí,17.91,38.0,Pro


# Analizamos los datos del fichero 'Seguimiento'

In [45]:
df_seguimiento = pd.read_csv('../data/Seguimiento.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')

In [46]:
df_seguimiento

,seguimiento_id,lead_id,contacto_id,curso_final_id,curso_final_nombre,comercial,closer,num_interacciones,fecha_ultima_interaccion,tipo_ultima_interaccion,...,modalidad_final,fecha_matricula,fecha_inicio_curso,motivo_no_conversion,probabilidad_retorno,valoracion_experiencia,feedback_proceso,irrelevante_1,irrelevante_2,irrelevante_3
0,a14d29da-b93d-47df-8ac9-0962c20bfd31,5b20df2f-49d1-49f2-bff4-e68e112ff6b4,c34d0149-83b5-4f88-9bac-e1c5eabaedef,87c56ad1,Data Science,Ana García,María López,3.0,2025-03-11,Email de seguimiento,...,Híbrido,2025-03-16,2025-05-19,NaN,35.0,10.0,Proceso muy ágil y profesional,FALSO,FALSO,FALSO
1,e87ebac7-796c-4ca3-89dc-c019fdf511ac,e40d99b0-a58c-4a6d-9779-d4e9d7f1f83a,1ccfbbab-4180-40ee-b9ed-0341a7e0660b,87c56ad1,Data Science,Carlos Martínez,NaN,2.0,2024-10-01,Email de seguimiento,...,NaN,NaN,NaN,El contenido del curso no se ajusta a sus nece...,45.0,4.0,Me gustaría más flexibilidad en los horarios,FALSO,VERDADERO,VERDADERO
2,4c118cf2-76a6-4b15-94f9-292c348e2c83,e6f9dc7e-6c56-4a4e-b8bb-c61f2f018301,30ceed35-ba2b-44e8-b481-a82e0642e90f,6fa49962,Desarrollo Web Full Stack,Sofía Fernández,David Sánchez,2.0,2025-01-14,Invitación a evento,...,Online,2025-01-15,2025-06-09,NaN,35.0,10.0,Proceso muy ágil y profesional,FALSO,VERDADERO,FALSO
3,694daba2-cc29-4593-9c5d-b3c2a28f4e09,8808cd20-a01b-41b1-8cbb-22fe18f1314b,ec61ce8d-1884-4935-9d78-f9ca89b48b63,6fa49962,Desarrollo Web Full Stack,Carlos Martínez,Roberto Fernández,4.0,2025-04-29,Asesoramiento financiero,...,Online,2025-04-30,2025-06-09,NaN,65.0,9.0,El asesoramiento ha sido clave para mi decisión,FALSO,FALSO,VERDADERO
4,4d463220-3f2f-4d08-a357-6fc54b941779,f17f1eb6-babe-44c4-a08c-2be3d6498319,6acf7adc-358b-41c3-bd9a-f60211c13765,6ae1ccdb,Ciberseguridad,Ana García,David Sánchez,3.0,2025-04-05,Llamada de seguimiento,...,Presencial,2025-04-06,2025-06-28,NaN,44.0,7.0,Me ha gustado mucho el trato personalizado,VERDADERO,VERDADERO,VERDADERO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104598,291b22b8-1827-43ad-b955-f87cfab0d79b,3a0867be-a144-46b3-878c-e8f97df778e3,a834d214-23fc-4f4d-aba1-51d52c1e4992,87C56AD1,Data Science,Sofía Fernández,Roberto Fernández,NaN,2025-02-11,Email de seguimiento,...,Híbrido,2025-02-15,2025-05-19,NaN,18.645.505.080.479.900,-18.491.355.140.438.000,INFORMACIÓN CLARA Y DETALLADA,VERDADERO,FALSO,FALSO
104599,d72f93d7-f5ca-42fb-906c-58fb4634bce8,C0D0D2C7-5272-48D5-B78E-E5431830B723,CB88633C-9952-49EB-8D48-CEC03895A678,A596A57E,Marketing Digital,ELENA RODRÍGUEZ,NaN,NaN,2024-12-20,ASESORAMIENTO FINANCIERO,...,NaN,NaN,NAN,Dejó de responder a los intentos de contacto,15.070.823.308.342.500,12.713.967.958.716.800,No he recibido suficiente información técnica ...,VERDADERO,VERDADERO,VERDADERO
104600,36540ed2-fab0-4853-b881-f7f7925aa087,a45c656a-ac50-4753-ba45-64eea5e04143,8f334ab3-9503-4cd9-b53a-6e1eb9f81774,87C56AD1,Data science,Sofía Fernández,ISABEL MARTÍNEZ,-18.979.244.950.026.900,2024-10-25,LLAMADA CON COORDINADOR ACADÉMICO,...,Online,2024-10-29,2025-05-19,Nan,12.894.621.286.182.300,NaN,El asesoramiento ha sido clave para mi decisión,FALSO,FALSO,FALSO
104601,AF1991FC-52C1-45F4-9453-831EDA23AD48,20f4886c-1ad4-4644-a574-493d6bf42491,C092df60-97e3-482e-86f7-9fd591513cd5,8E6D50AE,UX/UI DESIGN,Laura Sánchez,Isabel Martínez,7.809.034.206.133.110,2024-08-12,LLAMADA CON COORDINADOR ACADÉMICO,...,Híbrido,2024-08-15,2025-05-26,NAN,-11.879.826.272.233.000,NaN,INFORMACIÓN CLARA Y DETALLADA,FALSO,FALSO,VERDADERO


In [47]:
df_seguimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104603 entries, 0 to 104602
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   seguimiento_id            104603 non-null  object
 1   lead_id                   104603 non-null  object
 2   contacto_id               104603 non-null  object
 3   curso_final_id            104603 non-null  object
 4   curso_final_nombre        104603 non-null  object
 5   comercial                 104603 non-null  object
 6   closer                    55881 non-null   object
 7   num_interacciones         102370 non-null  object
 8   fecha_ultima_interaccion  104603 non-null  object
 9   tipo_ultima_interaccion   104603 non-null  object
 10  duracion_proceso_dias     103789 non-null  object
 11  estado_final              104603 non-null  object
 12  conversion                104603 non-null  object
 13  cambio_curso              104603 non-null  object
 14  desc

In [48]:
null_counts = df_seguimiento.isnull().sum()
null_counts


seguimiento_id                  0
lead_id                         0
contacto_id                     0
curso_final_id                  0
curso_final_nombre              0
comercial                       0
closer                      48722
num_interacciones            2233
fecha_ultima_interaccion        0
tipo_ultima_interaccion         0
duracion_proceso_dias         814
estado_final                    0
conversion                      0
cambio_curso                    0
descuento_aplicado          52820
promocion_aplicada          49483
precio_final                52805
metodo_pago                 49596
financiacion                49395
modalidad_final             49321
fecha_matricula             49416
fecha_inicio_curso          49134
motivo_no_conversion        50229
probabilidad_retorno         1535
valoracion_experiencia       3066
feedback_proceso            11424
irrelevante_1                   0
irrelevante_2                   0
irrelevante_3                   0
dtype: int64

In [49]:
columns_to_drop = ['irrelevante_1', 'irrelevante_2', 'irrelevante_3']
df_seguimiento = df_seguimiento.drop(columns=columns_to_drop)

In [50]:
date_columns = ['fecha_ultima_interaccion', 'fecha_matricula', 'fecha_inicio_curso']
for col in date_columns:
    df_seguimiento[col] = pd.to_datetime(df_seguimiento[col], errors='coerce')

In [51]:
df_seguimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104603 entries, 0 to 104602
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   seguimiento_id            104603 non-null  object        
 1   lead_id                   104603 non-null  object        
 2   contacto_id               104603 non-null  object        
 3   curso_final_id            104603 non-null  object        
 4   curso_final_nombre        104603 non-null  object        
 5   comercial                 104603 non-null  object        
 6   closer                    55881 non-null   object        
 7   num_interacciones         102370 non-null  object        
 8   fecha_ultima_interaccion  104603 non-null  datetime64[ns]
 9   tipo_ultima_interaccion   104603 non-null  object        
 10  duracion_proceso_dias     103789 non-null  object        
 11  estado_final              104603 non-null  object        
 12  co

In [52]:
boolean_columns = ['conversion', 'cambio_curso', 'financiacion']
for col in boolean_columns:
    df_seguimiento[col] = df_seguimiento[col].astype(bool)


In [53]:
numeric_columns = ['num_interacciones', 'duracion_proceso_dias', 'descuento_aplicado', 'precio_final', 'probabilidad_retorno', 'valoracion_experiencia']
for col in numeric_columns:
    df_seguimiento[col] = df_seguimiento[col].astype(str)
    df_seguimiento[col] = df_seguimiento[col].str.strip()
    df_seguimiento[col] = df_seguimiento[col].replace(['', 'nan', 'null', 'None'], np.nan)
    df_seguimiento[col] = pd.to_numeric(df_seguimiento[col], errors='coerce')


In [54]:
df_seguimiento['estado_final'] = df_seguimiento['estado_final'].str.strip().str.title()


In [55]:
print(df_seguimiento['estado_final'].value_counts())


estado_final
Matriculado                            52747
Perdido - No Responde                  13399
Perdido - Datos Incorrectos            10594
No Interesado - Contenido               8486
No Interesado - Precio                  5329
No Interesado - Sin Especificar         4437
No Interesado - Tiempo                  3894
No Interesado - Otra Escuela            3208
No Interesado - Otro Curso              1887
En Seguimiento - Media Probabilidad      281
En Seguimiento - Baja Probabilidad       248
En Seguimiento - Alta Probabilidad        93
Name: count, dtype: int64


In [56]:
df_seguimiento['metodo_pago'] = df_seguimiento['metodo_pago'].str.strip().str.title()

In [57]:
print(df_seguimiento['metodo_pago'].value_counts())

metodo_pago
Financiación En 12 Meses    14790
Financiación En 6 Meses     11598
Financiación En 3 Meses     11308
Tarjeta De Crédito           5054
Transferencia Bancaria       5034
Paypal                       4963
Nan                          2260
Name: count, dtype: int64


In [58]:
df_seguimiento['modalidad_final'] = df_seguimiento['modalidad_final'].str.strip().str.title()

In [59]:
print(df_seguimiento['modalidad_final'].value_counts())

modalidad_final
Online        27241
Presencial    16470
Híbrido        9036
Nan            2535
Name: count, dtype: int64


In [60]:
nombre_columns = ['comercial', 'closer']
for col in nombre_columns:
    df_seguimiento[col] = df_seguimiento[col].str.strip().str.title()

In [61]:
numeric_columns_for_validation = ['num_interacciones', 'duracion_proceso_dias', 'descuento_aplicado', 'precio_final', 'probabilidad_retorno', 'valoracion_experiencia']
for col in numeric_columns_for_validation:
    negative_count = df_seguimiento[df_seguimiento[col] < 0].shape[0]
    if negative_count > 0:
        df_seguimiento.loc[df_seguimiento[col] < 0, col] = np.nan

In [62]:
initial_rows = len(df_seguimiento)
df_seguimiento = df_seguimiento.drop_duplicates(subset=['seguimiento_id'], keep='first')
final_rows = len(df_seguimiento)
duplicates_removed = initial_rows - final_rows

# Analizamos los datos del fichero 'Primer contacto'

In [63]:
df_contacto = pd.read_csv('../data/Primer contacto.csv',
                          sep=';',
                          header=0,
                          index_col=False,
                          encoding='utf-8')

In [64]:
df_contacto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104755 entries, 0 to 104754
Data columns (total 24 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   contacto_id                       104755 non-null  object
 1   lead_id                           104755 non-null  object
 2   fecha_contacto                    104755 non-null  object
 3   hora_contacto                     104755 non-null  object
 4   medio_contacto                    104755 non-null  object
 5   comercial                         104755 non-null  object
 6   estado_contacto                   104755 non-null  object
 7   duracion_contacto_min             103024 non-null  object
 8   nivel_interes                     102797 non-null  object
 9   objeciones_principales            104755 non-null  object
 10  notas_adicionales                 104755 non-null  object
 11  interes_otros_cursos              104755 non-null  object
 12  ar

In [65]:
datos_nulos = df_contacto.isnull().sum()

In [66]:
porcentaje = (datos_nulos / len(df_contacto)) * 100
porcentaje

contacto_id                          0.000000
lead_id                              0.000000
fecha_contacto                       0.000000
hora_contacto                        0.000000
medio_contacto                       0.000000
comercial                            0.000000
estado_contacto                      0.000000
duracion_contacto_min                1.652427
nivel_interes                        1.869123
objeciones_principales               0.000000
notas_adicionales                    0.000000
interes_otros_cursos                 0.000000
areas_interes_adicional              0.000000
presupuesto_confirmado              24.940098
preferencia_fecha_inicio             0.000000
preferencia_financiacion             0.000000
disponibilidad_horaria_detallada     0.000000
conocimiento_competencia             0.000000
referenciado_por                     0.000000
proxima_accion                       0.000000
fecha_proxima_accion                11.098277
irrelevante_1                     

In [67]:
df_contacto = df_contacto.drop(columns=['irrelevante_1', 'irrelevante_2', 'irrelevante_3'])


In [68]:
text_columns = ['medio_contacto', 'comercial', 'estado_contacto', 'objeciones_principales', 
                'notas_adicionales', 'interes_otros_cursos', 'areas_interes_adicional',
                'preferencia_fecha_inicio', 'preferencia_financiacion', 'disponibilidad_horaria_detallada',
                'conocimiento_competencia', 'referenciado_por', 'proxima_accion']
for col in text_columns:
    df_contacto[col] = df_contacto[col].astype(str).str.strip()
    df_contacto[col] = df_contacto[col].str.replace(r'\s+', ' ', regex=True)
    df_contacto[col] = df_contacto[col].replace(['null', 'undefined', 'nan', 'n/a', 'na', 'none', 'nan'], np.nan)
 


In [69]:
df_contacto['fecha_contacto'] = pd.to_datetime(df_contacto['fecha_contacto'], errors='coerce')
df_contacto['fecha_proxima_accion'] = pd.to_datetime(df_contacto['fecha_proxima_accion'], errors='coerce')
df_contacto['hora_contacto'] = pd.to_datetime(df_contacto['hora_contacto'], errors='coerce').dt.strftime('%H:%M')


/var/folders/dy/g93lqr2159vf7_9t6j9sfgc80000gn/T/ipykernel_22832/1405710479.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_contacto['hora_contacto'] = pd.to_datetime(df_contacto['hora_contacto'], errors='coerce').dt.strftime('%H:%M')


In [70]:
df_contacto['duracion_contacto_min'] = pd.to_numeric(df_contacto['duracion_contacto_min'], errors='coerce')
df_contacto['nivel_interes'] = pd.to_numeric(df_contacto['nivel_interes'], errors='coerce')
df_contacto['presupuesto_confirmado'] = pd.to_numeric(df_contacto['presupuesto_confirmado'], errors='coerce')

In [71]:
df_contacto['duracion_contacto_min'].fillna(df_contacto['duracion_contacto_min'].median(), inplace=True)
df_contacto['nivel_interes'].fillna(df_contacto['nivel_interes'].median(), inplace=True)

/var/folders/dy/g93lqr2159vf7_9t6j9sfgc80000gn/T/ipykernel_22832/1639838447.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_contacto['duracion_contacto_min'].fillna(df_contacto['duracion_contacto_min'].median(), inplace=True)
/var/folders/dy/g93lqr2159vf7_9t6j9sfgc80000gn/T/ipykernel_22832/1639838447.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the

In [72]:
columnas_rellenar = ['presupuesto_confirmado', 'medio_contacto', 'comercial', 'estado_contacto', 'objeciones_principales', 
                    'notas_adicionales', 'interes_otros_cursos', 'areas_interes_adicional',
                    'preferencia_fecha_inicio', 'preferencia_financiacion', 'disponibilidad_horaria_detallada',
                    'conocimiento_competencia', 'referenciado_por', 'proxima_accion']

for col in columnas_rellenar:
    df_contacto[col].fillna('No especificado', inplace=True)

/var/folders/dy/g93lqr2159vf7_9t6j9sfgc80000gn/T/ipykernel_22832/3798485199.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_contacto[col].fillna('No especificado', inplace=True)
/var/folders/dy/g93lqr2159vf7_9t6j9sfgc80000gn/T/ipykernel_22832/3798485199.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No especificado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_contacto[col].fillna('No 

In [73]:
df_contacto = df_contacto.drop_duplicates(keep='first')

In [74]:
df_contacto['duracion_contacto_min'] = df_contacto['duracion_contacto_min'].astype(int)
df_contacto['nivel_interes'] = df_contacto['nivel_interes'].astype(int)

In [75]:
df_contacto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102302 entries, 0 to 102301
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   contacto_id                       102302 non-null  object        
 1   lead_id                           102302 non-null  object        
 2   fecha_contacto                    102302 non-null  datetime64[ns]
 3   hora_contacto                     102302 non-null  object        
 4   medio_contacto                    102302 non-null  object        
 5   comercial                         102302 non-null  object        
 6   estado_contacto                   102302 non-null  object        
 7   duracion_contacto_min             102302 non-null  int64         
 8   nivel_interes                     102302 non-null  int64         
 9   objeciones_principales            102302 non-null  object        
 10  notas_adicionales                 102

In [76]:
df_contacto

,contacto_id,lead_id,fecha_contacto,hora_contacto,medio_contacto,comercial,estado_contacto,duracion_contacto_min,nivel_interes,objeciones_principales,...,interes_otros_cursos,areas_interes_adicional,presupuesto_confirmado,preferencia_fecha_inicio,preferencia_financiacion,disponibilidad_horaria_detallada,conocimiento_competencia,referenciado_por,proxima_accion,fecha_proxima_accion
0,c34d0149-83b5-4f88-9bac-e1c5eabaedef,5b20df2f-49d1-49f2-bff4-e68e112ff6b4,2025-03-05,09:15,Email,Ana García,Contactado - Solicita más información,2,3,Falta de tiempo,...,Desarrollo Web Full Stack,Emprendimiento,5811.0,En más de 3 meses,Financiación bancaria,Disponible a partir de las 20:00,No conoce otras escuelas,No aplica,Programar entrevista con coordinador académico,2025-03-12
1,1ccfbbab-4180-40ee-b9ed-0341a7e0660b,e40d99b0-a58c-4a6d-9779-d4e9d7f1f83a,2024-09-27,11:30,Email,Carlos Martínez,Contactado - Solicita más información,2,2,Dudas sobre contenido,...,No aplica,No especificado,4343.0,En más de 3 meses,Financiación bancaria,Disponible entre 8:00 y 14:00,Comparando con 1-2 competidores,No aplica,Llamar en una semana,2024-10-06
2,30ceed35-ba2b-44e8-b481-a82e0642e90f,e6f9dc7e-6c56-4a4e-b8bb-c61f2f018301,2025-01-02,17:45,Llamada telefónica,Sofía Fernández,Contactado - Solicita más información,30,3,Necesita consultar,...,Desarrollo Web Full Stack,No especificado,3020.0,Lo antes posible,Pago fraccionado,Disponible a partir de las 20:00,No conoce otras escuelas,No aplica,Derivar a asesor financiero,2025-01-08
3,ec61ce8d-1884-4935-9d78-f9ca89b48b63,8808cd20-a01b-41b1-8cbb-22fe18f1314b,2025-04-17,19:45,Email,Carlos Martínez,Contactado - Solicita más información,2,5,Precio elevado,...,"Marketing Digital, Data Science",IoT,5615.0,En los próximos 3 meses,Pago fraccionado,Disponible Sábados entre 11:00 y 19:00,No conoce otras escuelas,No aplica,Agendar segunda llamada,2025-04-22
4,6acf7adc-358b-41c3-bd9a-f60211c13765,f17f1eb6-babe-44c4-a08c-2be3d6498319,2025-03-18,12:15,Email,Ana García,Contactado - Solicita más información,4,2,Dudas sobre modalidad,...,No aplica,No especificado,2800.0,Lo antes posible,Financiación bancaria,Disponible Sábados y Domingos entre 12:00 y 18:00,No conoce otras escuelas,Compañero de trabajo,Invitar a webinar gratuito,2025-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102297,4e898ae1-01d8-4cbb-a690-f026af6abc90,c2c7adad-d45e-4c2e-acfc-e55f57d509f5,2025-03-21,15:45,Email,Laura Sánchez,Contactado - Solicita más información,1,5,Dudas sobre contenido,...,UX/UI Design,No especificado,4404.0,Lo antes posible,Pago fraccionado,Disponible entre 17:00 y 20:00,Evaluando múltiples opciones,No aplica,Programar entrevista con coordinador académico,2025-03-23
102298,7bb1b8b4-d4a1-4505-afcf-96e068c7b704,e1b9a6f0-33ff-4270-8f01-716d62941a65,2024-08-30,10:00,Llamada telefónica,Javier López,Contactado - Interesado,7,2,Comparando con competencia,...,No aplica,No especificado,2768.0,Lo antes posible,Pago fraccionado,Disponible Domingos entre 10:00 y 17:00,Ya ha hecho formaciones similares,No aplica,Invitar a Open Day,2024-09-04
102299,a7eebc0e-d05e-499d-b7ba-05217fbb39b5,cea39cac-1c75-4e2a-9b5d-0c15b36c860d,2025-03-04,17:45,WhatsApp,Ana García,Contactado - Solicita más información,4,3,Falta de tiempo,...,Data Analytics,Realidad aumentada/virtual,5159.0,En los próximos 3 meses,Pago fraccionado,Disponible entre 9:00 y 12:00,Evaluando múltiples opciones,No aplica,Invitar a webinar gratuito,2025-03-18
102300,fc58f7ba-cf68-45fc-8bf8-364aa474113d,c071e4e2-b312-4b4b-9ef7-65ee47c78ff1,2025-01-05,14:45,Llamada telefónica,Miguel González,Contactado - No interesado,25,2,Dudas sobre contenido,...,No aplica,No especificado,6520.0,En los próximos 3 meses,No especificado,Disponible entre 16:00 y 19:00,No conoce otras escuelas,No aplica,Cerrar lead (no interesado),NaT


In [77]:
df_leads.to_csv('../data_clean/leads_limpio.csv', sep=';', index=False)
df_cursos.to_csv('../data_clean/cursos_limpio.csv', sep=';', index=False)
df_seguimiento.to_csv('../data_clean/seguimiento_limpio.csv', sep=';', index=False)
df_contacto.to_csv('../data_clean/primer_contacto_limpio.csv', sep=';', index=False)

# Transformaciones necesarias para cargar los datos en sql.

In [78]:
df_leads = pd.read_csv('../data_clean/leads_limpio.csv', sep=';')
df_cursos = pd.read_csv('../data_clean/cursos_limpio.csv', sep=';')
df_seguimiento = pd.read_csv('../data_clean/seguimiento_limpio.csv', sep=';')
df_contacto = pd.read_csv('../data_clean/primer_contacto_limpio.csv', sep=';')

Asigno los mismos id a las 2 tablas para que no den error en sql

In [79]:
print("Cursos asociados a 181bd83f:")
print(df_leads[df_leads['curso_interes_id'].str.lower() == '181bd83f']['curso_interes_nombre'].value_counts())

Cursos asociados a 181bd83f:
curso_interes_nombre
IA Developer    14185
IA DEVELOPER      519
Ia developer      240
Name: count, dtype: int64


In [80]:
mapeo_data = {
    'id_antiguo': ['87c56ad1', 'c0e24ebc', '6fa49962', '6ae1ccdb', '8e6d50ae', 'a596a57e', '181bd83f'],
    'id_nuevo': ['33a93363', 'f514fe84', '546ad5a9', '3530776b', '0e567c6f', 'e77f35b7', 'f8c5f595'],
    'nombre_curso': ['Data Science', 'Data Analytics', 'Desarrollo Web Full Stack', 
                     'Ciberseguridad', 'UX/UI Design', 'Marketing Digital', 'IA Developer']
}

In [81]:
df_mapeo = pd.DataFrame(mapeo_data)
mapeo_dict = df_mapeo.set_index('id_antiguo')['id_nuevo'].to_dict()

In [82]:
cursos_final = df_cursos.rename(columns={
    'Curso ID': 'curso_id',
    'Nombre': 'nombre',
    'Modalidad': 'modalidad',
    'Duracion meses': 'duracion_meses',
    'Horas totales': 'horas_totales',
    'Precio base': 'precio_base',
    'Precio descuento': 'precio_descuento',
    'Tecnologias': 'tecnologias',
    'Nivel': 'nivel',
    'Requisitos previos': 'requisitos_previos',
    'Fecha inicio proxima': 'fecha_inicio_proxima',
    'Plazas disponibles': 'plazas_disponibles',
    'Rating': 'rating',
    'Empleabilidad': 'empleabilidad',
    'Descripción': 'descripcion',
    'Certificación oficial': 'certificacion_oficial'
})

In [83]:
cursos_final['certificacion_oficial'] = cursos_final['certificacion_oficial'].str.replace('Sí', 'Si', regex=False)

In [84]:
cursos_final['certificacion_oficial'] = cursos_final['certificacion_oficial'].map({'Si': True, 'No': False})

In [85]:
cursos_final['certificacion_oficial']

0     True
1    False
2    False
3     True
4     True
5     True
6     True
Name: certificacion_oficial, dtype: bool

In [86]:
cursos_final['certificacion_oficial'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7 entries, 0 to 6
Series name: certificacion_oficial
Non-Null Count  Dtype
--------------  -----
7 non-null      bool 
dtypes: bool(1)
memory usage: 139.0 bytes


In [87]:
cursos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   curso_id               7 non-null      object 
 1   nombre                 7 non-null      object 
 2   modalidad              7 non-null      object 
 3   duracion_meses         7 non-null      float64
 4   horas_totales          7 non-null      float64
 5   precio_base            7 non-null      float64
 6   precio_descuento       7 non-null      float64
 7   tecnologias            7 non-null      object 
 8   nivel                  7 non-null      object 
 9   requisitos_previos     7 non-null      object 
 10  fecha_inicio_proxima   7 non-null      object 
 11  plazas_disponibles     7 non-null      float64
 12  rating                 7 non-null      float64
 13  empleabilidad          7 non-null      float64
 14  url_dossier            7 non-null      object 
 15  descripcio

In [88]:
tecnologias_series = cursos_final['tecnologias'].str.split(', ').explode()
tecnologias_series = tecnologias_series.str.strip()
tecnologias_unicas = tecnologias_series.unique()
tecnologias_unicas = tecnologias_unicas[pd.notna(tecnologias_unicas)]

In [89]:
df_tecnologias = pd.DataFrame({
    'tecnologia_id': range(1, len(tecnologias_unicas) + 1),
    'nombre': tecnologias_unicas
})

In [90]:
df_tecnologias.value_counts().sum()

np.int64(38)

Compruebo que tengo la misma cantidad de ID's que de tecnologías (38)

In [91]:
df_tecnologias['tecnologia_id'].value_counts().sum()

np.int64(38)

In [92]:
df_leads['curso_interes_id'] = df_leads['curso_interes_id'].map(mapeo_dict)
df_leads['fecha_registro'] = pd.to_datetime(df_leads['fecha_registro'])

In [93]:
df_leads[['comprobaciónaleatoria', 'comprobaciónaleatoria2']]

,comprobaciónaleatoria,comprobaciónaleatoria2
0,VERDADERO,VERDADERO
1,VERDADERO,VERDADERO
2,FALSO,FALSO
3,VERDADERO,VERDADERO
4,VERDADERO,FALSO
...,...,...
105240,VERDADERO,VERDADERO
105241,FALSO,FALSO
105242,VERDADERO,FALSO
105243,VERDADERO,FALSO


In [94]:
df_leads

,lead_id,curso_interes_id,curso_interes_nombre,fecha_registro,fuente,campana,nombre,apellidos,email,telefono,...,horario_preferido,modalidad_preferida,comercial_asignado,lead_score,codigo,comprobaciónaleatoria,comprobaciónaleatoria2,grupo_edad,categoria_presupuesto,calidad_lead
0,5b20df2f-49d1-49f2-bff4-e68e112ff6b4,33a93363,Data Science,2025-03-04,Facebook Ads,FB_Promocion_Mayo,Feliciana,Cantón Amaya,feliciana.canton@hotmail.com,627680402.0,...,Noche,Híbrido,Ana García,45.0,3172,VERDADERO,VERDADERO,46-55,Alto,Medio
1,e40d99b0-a58c-4a6d-9779-d4e9d7f1f83a,33a93363,Data Science,2024-09-26,Google Ads,GG_Search_Cursos,Leonardo,Hierro Godoy,leonardo.hierro@gmail.com,693008687.0,...,Mañana,Online,Carlos Martínez,58.0,3251,VERDADERO,VERDADERO,46-55,Alto,Medio
2,e6f9dc7e-6c56-4a4e-b8bb-c61f2f018301,f514fe84,Data Analytics,2025-01-01,YouTube,YT_Testimonial,Maite,Calatayud Bonet,maite.calatayud@hotmail.com,748136760.0,...,Noche,Híbrido,Sofía Fernández,53.0,5619,FALSO,FALSO,26-35,Bajo,Medio
3,8808cd20-a01b-41b1-8cbb-22fe18f1314b,546ad5a9,Desarrollo Web Full Stack,2025-04-14,Email Marketing,Email_Promo,Telmo,Antón Andrés,telmo.anton@gmail.com,709572486.0,...,Fin de semana,Online,Carlos Martínez,77.0,9276,VERDADERO,VERDADERO,26-35,Alto,Alto
4,f17f1eb6-babe-44c4-a08c-2be3d6498319,3530776b,Ciberseguridad,2025-03-15,Referido,Referral_Alumnos,Isaías,Guerrero Vazquez,isaias.guerrero@hotmail.com,667403959.0,...,Fin de semana,Presencial,Ana García,66.0,6749,VERDADERO,FALSO,18-25,Medio,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105240,e07d5dce-456a-4718-ba3a-ffa1ccd64e55,33a93363,Data Science,2024-11-06,FACEBOOK ADS,FB_Career_2025,Luz,Calzada Camacho,luz.calzada@gmail.com,NaN,...,Tarde,PRESENCIAL,Carlos Martínez,NaN,5579,VERDADERO,VERDADERO,NaN,Bajo,NaN
105241,A42FDCBF-E951-4D38-B860-576F5FB7BA45,NaN,Data Science,2025-01-12,TIKTOK,TT_Challenge,Laura,Barberá Almagro,laura.barbera@gmail.com,NaN,...,Tarde,Presencial,Ana García,NaN,6270,FALSO,FALSO,NaN,Pro,NaN
105242,Dc5a9362-b4b8-40f3-a7f3-6f91996de995,f8c5f595,IA DEVELOPER,2024-10-24,Google ads,Gg_remarketing,Rafael,Espinosa Fuertes,rafael.espinosa@hotmail.com,NaN,...,Noche,Híbrido,Ana García,NaN,2068,VERDADERO,FALSO,NaN,Bajo,NaN
105243,Aaea5f50-707f-43b4-9278-eea79ff5b5f9,NaN,CIBERSEGURIDAD,2024-08-04,Google Ads,Gg_remarketing,Adelia,Pozuelo Blázquez,adelia.pozuelo@gmail.com,NaN,...,FLEXIBLE,Presencial,Miguel González,NaN,4358,VERDADERO,FALSO,NaN,Bajo,NaN


Quito columnas innecesarias para la tabla sql

In [95]:
df_leads = df_leads.drop(columns='codigo', errors='ignore')
df_leads = df_leads.drop(columns='comprobaciónaleatoria', errors='ignore')
df_leads = df_leads.drop(columns='comprobaciónaleatoria2', errors='ignore')
df_leads = df_leads.drop(columns='grupo_edad', errors='ignore')
df_leads = df_leads.drop(columns='categoria_presupuesto', errors='ignore')
df_leads = df_leads.drop(columns='calidad_lead', errors='ignore')

In [96]:
df_leads['curso_interes_id'] = df_leads['curso_interes_id'].str.lower()
df_seguimiento['curso_final_id'] = df_seguimiento['curso_final_id'].str.lower()

In [97]:
#df_leads['curso_interes_id'] = df_leads['curso_interes_id'].map(mapeo_dict)
df_leads['fecha_registro'] = pd.to_datetime(df_leads['fecha_registro'])
df_leads['telefono'] = df_leads['telefono'].replace('', np.nan)

In [98]:
df_contacto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102302 entries, 0 to 102301
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   contacto_id                       102302 non-null  object
 1   lead_id                           102302 non-null  object
 2   fecha_contacto                    102302 non-null  object
 3   hora_contacto                     102302 non-null  object
 4   medio_contacto                    102302 non-null  object
 5   comercial                         102302 non-null  object
 6   estado_contacto                   102302 non-null  object
 7   duracion_contacto_min             102302 non-null  int64 
 8   nivel_interes                     102302 non-null  int64 
 9   objeciones_principales            102302 non-null  object
 10  notas_adicionales                 102302 non-null  object
 11  interes_otros_cursos              102302 non-null  object
 12  ar

In [99]:
df_contacto['fecha_contacto'] = pd.to_datetime(df_contacto['fecha_contacto'])
df_contacto['fecha_proxima_accion'] = pd.to_datetime(df_contacto['fecha_proxima_accion'], errors='coerce')

In [100]:
def limpiar_presupuesto(valor):
    if pd.isna(valor) or valor == 'No especificado':
        return np.nan
    try:
        return float(valor)
    except:
        return np.nan

In [101]:
df_contacto['presupuesto_confirmado'] = df_contacto['presupuesto_confirmado'].apply(limpiar_presupuesto)

In [102]:
df_contacto['presupuesto_confirmado'].head(50)

0     5811.0
1     4343.0
2     3020.0
3     5615.0
4     2800.0
5     2580.0
6     2310.0
7        NaN
8     1986.0
9     5022.0
10    3053.0
11    3393.0
12       NaN
13    3771.0
14    2173.0
15    2457.0
16    4174.0
17    5716.0
18    6241.0
19    3621.0
20    2318.0
21       NaN
22       NaN
23       NaN
24    2503.0
25    2068.0
26    2662.0
27       NaN
28       NaN
29    2502.0
30    3114.0
31    5876.0
32    4378.0
33    2898.0
34    2364.0
35       NaN
36    2732.0
37    2514.0
38    5116.0
39    2898.0
40    2577.0
41    2634.0
42    1746.0
43       NaN
44    2000.0
45    5149.0
46       NaN
47    2024.0
48    1844.0
49       NaN
Name: presupuesto_confirmado, dtype: float64

In [103]:
df_contacto['interes_otros_cursos'] = df_contacto['interes_otros_cursos'].apply(lambda x: False if pd.isna(x) or str(x).lower() in ['no', 'no aplica', 'ninguno'] else True)

In [104]:
df_contacto['interes_otros_cursos']

0          True
1         False
2          True
3          True
4         False
          ...  
102297     True
102298    False
102299     True
102300    False
102301     True
Name: interes_otros_cursos, Length: 102302, dtype: bool

Normalizamos también los id en el df_seguimiento

In [105]:
df_seguimiento['curso_final_id'] = df_seguimiento['curso_final_id'].map(mapeo_dict)

In [106]:
df_seguimiento

,seguimiento_id,lead_id,contacto_id,curso_final_id,curso_final_nombre,comercial,closer,num_interacciones,fecha_ultima_interaccion,tipo_ultima_interaccion,...,precio_final,metodo_pago,financiacion,modalidad_final,fecha_matricula,fecha_inicio_curso,motivo_no_conversion,probabilidad_retorno,valoracion_experiencia,feedback_proceso
0,a14d29da-b93d-47df-8ac9-0962c20bfd31,5b20df2f-49d1-49f2-bff4-e68e112ff6b4,c34d0149-83b5-4f88-9bac-e1c5eabaedef,33a93363,Data Science,Ana García,María López,3.0,2025-03-11,Email de seguimiento,...,5018.0,Financiación En 12 Meses,True,Híbrido,2025-03-16,2025-05-19,NaN,35.0,10.0,Proceso muy ágil y profesional
1,e87ebac7-796c-4ca3-89dc-c019fdf511ac,e40d99b0-a58c-4a6d-9779-d4e9d7f1f83a,1ccfbbab-4180-40ee-b9ed-0341a7e0660b,33a93363,Data Science,Carlos Martínez,NaN,2.0,2024-10-01,Email de seguimiento,...,NaN,NaN,True,NaN,NaN,NaN,El contenido del curso no se ajusta a sus nece...,45.0,4.0,Me gustaría más flexibilidad en los horarios
2,4c118cf2-76a6-4b15-94f9-292c348e2c83,e6f9dc7e-6c56-4a4e-b8bb-c61f2f018301,30ceed35-ba2b-44e8-b481-a82e0642e90f,546ad5a9,Desarrollo Web Full Stack,Sofía Fernández,David Sánchez,2.0,2025-01-14,Invitación a evento,...,4896.0,Financiación En 6 Meses,True,Online,2025-01-15,2025-06-09,NaN,35.0,10.0,Proceso muy ágil y profesional
3,694daba2-cc29-4593-9c5d-b3c2a28f4e09,8808cd20-a01b-41b1-8cbb-22fe18f1314b,ec61ce8d-1884-4935-9d78-f9ca89b48b63,546ad5a9,Desarrollo Web Full Stack,Carlos Martínez,Roberto Fernández,4.0,2025-04-29,Asesoramiento financiero,...,4406.0,Financiación En 6 Meses,True,Online,2025-04-30,2025-06-09,NaN,65.0,9.0,El asesoramiento ha sido clave para mi decisión
4,4d463220-3f2f-4d08-a357-6fc54b941779,f17f1eb6-babe-44c4-a08c-2be3d6498319,6acf7adc-358b-41c3-bd9a-f60211c13765,3530776b,Ciberseguridad,Ana García,David Sánchez,3.0,2025-04-05,Llamada de seguimiento,...,2380.0,Financiación En 12 Meses,True,Presencial,2025-04-06,2025-06-28,NaN,44.0,7.0,Me ha gustado mucho el trato personalizado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102297,a6a3f372-5a4a-499e-8a46-267e82b324f4,c2c7adad-d45e-4c2e-acfc-e55f57d509f5,4e898ae1-01d8-4cbb-a690-f026af6abc90,0e567c6f,UX/UI Design,Laura Sánchez,David Sánchez,2.0,2025-03-31,Envío de contenido adicional,...,2589.0,Financiación En 6 Meses,True,Online,2025-04-07,2025-05-26,NaN,80.0,10.0,Me ha gustado mucho el trato personalizado
102298,5ee1a077-c53d-4841-9dc8-353f0c793566,e1b9a6f0-33ff-4270-8f01-716d62941a65,7bb1b8b4-d4a1-4505-afcf-96e068c7b704,0e567c6f,UX/UI Design,Javier López,Isabel Martínez,5.0,2024-09-14,WhatsApp,...,2445.0,Financiación En 3 Meses,True,Online,2024-09-19,2025-05-26,NaN,42.0,8.0,Proceso muy ágil y profesional
102299,3cfa163e-d1dd-4533-afc9-1b4f4c4a5a1b,cea39cac-1c75-4e2a-9b5d-0c15b36c860d,a7eebc0e-d05e-499d-b7ba-05217fbb39b5,f514fe84,Data Analytics,Ana García,Isabel Martínez,2.0,2025-03-12,Email de seguimiento,...,2120.0,Financiación En 6 Meses,True,Híbrido,2025-03-17,2025-05-28,NaN,60.0,7.0,Proceso muy ágil y profesional
102300,7a2f0e7d-9f05-4087-b5c6-7655cb0a79d3,c071e4e2-b312-4b4b-9ef7-65ee47c78ff1,fc58f7ba-cf68-45fc-8bf8-364aa474113d,33a93363,Data Science,Miguel González,María López,2.0,2025-01-11,Asesoramiento financiero,...,4723.0,Financiación En 3 Meses,True,Presencial,2025-01-14,2025-05-19,NaN,49.0,10.0,Me ha gustado mucho el trato personalizado


In [107]:
date_cols = ['fecha_ultima_interaccion', 'fecha_matricula', 'fecha_inicio_curso']
for col in date_cols:
    df_seguimiento[col] = pd.to_datetime(df_seguimiento[col], errors='coerce')

In [108]:
boolean_cols = ['conversion', 'cambio_curso', 'financiacion']
for col in boolean_cols:
    df_seguimiento[col] = df_seguimiento[col].map({
        'True': True, 'False': False, True: True, False: False
    })

In [109]:
df_seguimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102302 entries, 0 to 102301
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   seguimiento_id            102302 non-null  object        
 1   lead_id                   102302 non-null  object        
 2   contacto_id               102302 non-null  object        
 3   curso_final_id            102302 non-null  object        
 4   curso_final_nombre        102302 non-null  object        
 5   comercial                 102302 non-null  object        
 6   closer                    54042 non-null   object        
 7   num_interacciones         95699 non-null   float64       
 8   fecha_ultima_interaccion  102302 non-null  datetime64[ns]
 9   tipo_ultima_interaccion   102302 non-null  object        
 10  duracion_proceso_dias     100094 non-null  float64       
 11  estado_final              102302 non-null  object        
 12  co

In [110]:
numeric_cols = ['num_interacciones', 'duracion_proceso_dias', 'descuento_aplicado', 'precio_final', 'probabilidad_retorno', 'valoracion_experiencia']

for col in numeric_cols:
    df_seguimiento[col] = pd.to_numeric(df_seguimiento[col], errors='coerce')

In [111]:
cursos_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   curso_id               7 non-null      object 
 1   nombre                 7 non-null      object 
 2   modalidad              7 non-null      object 
 3   duracion_meses         7 non-null      float64
 4   horas_totales          7 non-null      float64
 5   precio_base            7 non-null      float64
 6   precio_descuento       7 non-null      float64
 7   tecnologias            7 non-null      object 
 8   nivel                  7 non-null      object 
 9   requisitos_previos     7 non-null      object 
 10  fecha_inicio_proxima   7 non-null      object 
 11  plazas_disponibles     7 non-null      float64
 12  rating                 7 non-null      float64
 13  empleabilidad          7 non-null      float64
 14  url_dossier            7 non-null      object 
 15  descripcio

In [112]:
cursos_final = cursos_final.drop(columns='Descuento_porcentaje', errors='ignore')
cursos_final = cursos_final.drop(columns='Horas_por_mes', errors='ignore')  
cursos_final = cursos_final.drop(columns='Categoria_precio', errors='ignore')
cursos_final = cursos_final.drop(columns='tecnologias', errors='ignore')

In [113]:

cursos_final['duracion_meses'] = cursos_final['duracion_meses'].astype('Int64')
cursos_final['horas_totales'] = cursos_final['horas_totales'].astype('Int64')
cursos_final['plazas_disponibles'] = cursos_final['plazas_disponibles'].astype('Int64')
cursos_final['fecha_inicio_proxima'] = pd.to_datetime(cursos_final['fecha_inicio_proxima'], errors='coerce')

### Comprobamos y modificamos las columnas para que se adapten al formato de nuestra estructura sql

In [114]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105245 entries, 0 to 105244
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lead_id               105245 non-null  object        
 1   curso_interes_id      101199 non-null  object        
 2   curso_interes_nombre  105245 non-null  object        
 3   fecha_registro        105245 non-null  datetime64[ns]
 4   fuente                105245 non-null  object        
 5   campana               105245 non-null  object        
 6   nombre                105245 non-null  object        
 7   apellidos             105245 non-null  object        
 8   email                 105245 non-null  object        
 9   telefono              99260 non-null   float64       
 10  edad                  96513 non-null   float64       
 11  ciudad                105245 non-null  object        
 12  provincia             105245 non-null  object        
 13 

In [115]:
df_leads['edad'] = df_leads['edad'].astype('Int64')

In [116]:
df_leads['urgencia_formacion'] = df_leads['urgencia_formacion'].astype('string')

In [117]:
df_leads['urgencia_formacion'] = df_leads['urgencia_formacion'].str.replace('.0', '', regex=False)

In [118]:
df_contacto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102302 entries, 0 to 102301
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   contacto_id                       102302 non-null  object        
 1   lead_id                           102302 non-null  object        
 2   fecha_contacto                    102302 non-null  datetime64[ns]
 3   hora_contacto                     102302 non-null  object        
 4   medio_contacto                    102302 non-null  object        
 5   comercial                         102302 non-null  object        
 6   estado_contacto                   102302 non-null  object        
 7   duracion_contacto_min             102302 non-null  int64         
 8   nivel_interes                     102302 non-null  int64         
 9   objeciones_principales            102302 non-null  object        
 10  notas_adicionales               

In [119]:
df_contacto['hora_contacto']= pd.to_datetime(df_contacto['hora_contacto'], format='%H:%M').dt.time

In [120]:
df_contacto['nivel_interes'] = df_contacto['nivel_interes'].astype('string')

## Establecemos un patrón para la preferencia de los posibles alumnos.

In [121]:
df_contacto['preferencia_fecha_inicio']= df_contacto['preferencia_fecha_inicio'].str.lower()

In [122]:
mapa_fechas = {
   'lo antes posible': datetime.now().date() + timedelta(days=7),
   'en los próximos 3 meses': datetime.now().date() + timedelta(days=90),
   'en más de 3 meses': datetime.now().date() + timedelta(days=120),
   'aún no lo tiene claro': datetime.now().date() + timedelta(days=180),
   'no aplica': None
}

df_contacto['preferencia_fecha_inicio'] = df_contacto['preferencia_fecha_inicio'].map(mapa_fechas)

In [123]:
df_contacto['preferencia_fecha_inicio'] = pd.to_datetime(df_contacto['preferencia_fecha_inicio'])

In [124]:
df_contacto['preferencia_fecha_inicio']

0        2025-09-22
1        2025-09-22
2        2025-06-01
3        2025-08-23
4        2025-06-01
            ...    
102297   2025-06-01
102298   2025-06-01
102299   2025-08-23
102300   2025-08-23
102301   2025-06-01
Name: preferencia_fecha_inicio, Length: 102302, dtype: datetime64[ns]

In [125]:
df_seguimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102302 entries, 0 to 102301
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   seguimiento_id            102302 non-null  object        
 1   lead_id                   102302 non-null  object        
 2   contacto_id               102302 non-null  object        
 3   curso_final_id            102302 non-null  object        
 4   curso_final_nombre        102302 non-null  object        
 5   comercial                 102302 non-null  object        
 6   closer                    54042 non-null   object        
 7   num_interacciones         95699 non-null   float64       
 8   fecha_ultima_interaccion  102302 non-null  datetime64[ns]
 9   tipo_ultima_interaccion   102302 non-null  object        
 10  duracion_proceso_dias     100094 non-null  float64       
 11  estado_final              102302 non-null  object        
 12  co

In [126]:
df_seguimiento['num_interacciones'] = df_seguimiento['num_interacciones'].astype('Int64')

In [127]:
df_seguimiento['duracion_proceso_dias'] = df_seguimiento['duracion_proceso_dias'].astype('Int64')

In [128]:
df_seguimiento['valoracion_experiencia'] = df_seguimiento['valoracion_experiencia'].astype('Int64')

In [129]:
df_seguimiento['financiacion'] = df_seguimiento['financiacion'].astype('string')

In [130]:
df_seguimiento['probabilidad_retorno'] = pd.cut(
    df_seguimiento['probabilidad_retorno'], 
    bins=[0, 30, 70, 100], 
    labels=['bajo', 'medio', 'alto'], 
    include_lowest=True
)

In [131]:
df_seguimiento['probabilidad_retorno']

0         medio
1         medio
2         medio
3         medio
4         medio
          ...  
102297     alto
102298    medio
102299    medio
102300    medio
102301    medio
Name: probabilidad_retorno, Length: 102302, dtype: category
Categories (3, object): ['bajo' < 'medio' < 'alto']

In [132]:
df_tecnologias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tecnologia_id  38 non-null     int64 
 1   nombre         38 non-null     object
dtypes: int64(1), object(1)
memory usage: 740.0+ bytes


In [133]:
cursos_final.head()

,curso_id,nombre,modalidad,duracion_meses,horas_totales,precio_base,precio_descuento,nivel,requisitos_previos,fecha_inicio_proxima,plazas_disponibles,rating,empleabilidad,url_dossier,descripcion,certificacion_oficial
0,f514fe84,Data Analytics,Híbrido,7,364,4018.0,3426.0,Avanzado,Conocimientos básicos de ofimática,2025-06-01,22,4.2,87.0,https://escuelaformacion.edu/dossier/data-anal...,Conviértete en un profesional del análisis de ...,True
1,546ad5a9,Desarrollo Web Full Stack,Presencial,11,407,6325.0,5701.0,Principiante,"Ninguno, se parte desde cero",2025-06-09,16,4.8,87.0,https://escuelaformacion.edu/dossier/desarroll...,"Aprende a crear aplicaciones web completas, de...",False
2,f8c5f595,IA Developer,Presencial,4,144,2180.0,1784.0,Intermedio,"Conocimientos de programación, preferiblemente...",2025-06-07,30,4.4,95.0,https://escuelaformacion.edu/dossier/ia-develo...,Especialízate en el desarrollo de soluciones b...,False
3,3530776b,Ciberseguridad,Presencial,5,190,2175.0,1985.0,Avanzado,Conocimientos básicos de redes y sistemas,2025-07-18,23,4.7,88.0,https://escuelaformacion.edu/dossier/cibersegu...,Protege sistemas y redes contra vulnerabilidad...,True
4,0e567c6f,UX/UI Design,Híbrido,9,369,4104.0,3284.0,Principiante,"Interés por el diseño, no se requiere experien...",2025-07-15,30,4.1,76.0,https://escuelaformacion.edu/dossier/ux/ui-des...,Diseña interfaces intuitivas y experiencias de...,True


In [134]:
cursos_final['curso_id'] = pd.factorize(cursos_final['curso_id'])[0] + 1

In [135]:
cursos_final

,curso_id,nombre,modalidad,duracion_meses,horas_totales,precio_base,precio_descuento,nivel,requisitos_previos,fecha_inicio_proxima,plazas_disponibles,rating,empleabilidad,url_dossier,descripcion,certificacion_oficial
0,1,Data Analytics,Híbrido,7,364,4018.0,3426.0,Avanzado,Conocimientos básicos de ofimática,2025-06-01,22,4.2,87.0,https://escuelaformacion.edu/dossier/data-anal...,Conviértete en un profesional del análisis de ...,True
1,2,Desarrollo Web Full Stack,Presencial,11,407,6325.0,5701.0,Principiante,"Ninguno, se parte desde cero",2025-06-09,16,4.8,87.0,https://escuelaformacion.edu/dossier/desarroll...,"Aprende a crear aplicaciones web completas, de...",False
2,3,IA Developer,Presencial,4,144,2180.0,1784.0,Intermedio,"Conocimientos de programación, preferiblemente...",2025-06-07,30,4.4,95.0,https://escuelaformacion.edu/dossier/ia-develo...,Especialízate en el desarrollo de soluciones b...,False
3,4,Ciberseguridad,Presencial,5,190,2175.0,1985.0,Avanzado,Conocimientos básicos de redes y sistemas,2025-07-18,23,4.7,88.0,https://escuelaformacion.edu/dossier/cibersegu...,Protege sistemas y redes contra vulnerabilidad...,True
4,5,UX/UI Design,Híbrido,9,369,4104.0,3284.0,Principiante,"Interés por el diseño, no se requiere experien...",2025-07-15,30,4.1,76.0,https://escuelaformacion.edu/dossier/ux/ui-des...,Diseña interfaces intuitivas y experiencias de...,True
5,6,Data Science,Online,5,250,2200.0,1828.0,Intermedio,Conocimientos básicos de programación y estadí...,2025-07-26,17,4.4,94.0,https://escuelaformacion.edu/dossier/data-scie...,Domina la ciencia de datos para extraer insigh...,True
6,7,Marketing Digital,Presencial,11,418,5951.0,4885.0,Principiante,"Ninguno, se parte desde cero",2025-05-25,23,4.8,85.0,https://escuelaformacion.edu/dossier/marketing...,Desarrolla estrategias de marketing online efe...,True


In [136]:
mapeo_data2 = {
    'id_antiguo': ['33a93363', 'f514fe84', '546ad5a9', '3530776b', '0e567c6f', 'e77f35b7', 'f8c5f595'],
    'id_nuevo': ['6', '1', '2', '4', '5', '7', '3'],
    'nombre_curso': ['Data Science', 'Data Analytics', 'Desarrollo Web Full Stack', 
                     'Ciberseguridad', 'UX/UI Design', 'Marketing Digital', 'IA Developer']
}

In [137]:
df_mapeo2 = pd.DataFrame(mapeo_data2)
mapeo_dict2 = df_mapeo2.set_index('id_antiguo')['id_nuevo'].to_dict()

In [138]:
df_leads['curso_interes_id'] = df_leads['curso_interes_id'].map(mapeo_dict2)
df_seguimiento['curso_final_id'] = df_seguimiento['curso_final_id'].map(mapeo_dict2)


In [139]:
df_leads

,lead_id,curso_interes_id,curso_interes_nombre,fecha_registro,fuente,campana,nombre,apellidos,email,telefono,...,nivel_estudios,situacion_laboral,objetivo_profesional,presupuesto_estimado,urgencia_formacion,dispositivo_registro,horario_preferido,modalidad_preferida,comercial_asignado,lead_score
0,5b20df2f-49d1-49f2-bff4-e68e112ff6b4,6,Data Science,2025-03-04,Facebook Ads,FB_Promocion_Mayo,Feliciana,Cantón Amaya,feliciana.canton@hotmail.com,627680402.0,...,Formación Profesional,En búsqueda activa,Desarrollo profesional,5741.00,2,Móvil,Noche,Híbrido,Ana García,45.0
1,e40d99b0-a58c-4a6d-9779-d4e9d7f1f83a,6,Data Science,2024-09-26,Google Ads,GG_Search_Cursos,Leonardo,Hierro Godoy,leonardo.hierro@gmail.com,693008687.0,...,Bachillerato,Empleado tiempo parcial,Desarrollo profesional,5958.00,3,Móvil,Mañana,Online,Carlos Martínez,58.0
2,e6f9dc7e-6c56-4a4e-b8bb-c61f2f018301,1,Data Analytics,2025-01-01,YouTube,YT_Testimonial,Maite,Calatayud Bonet,maite.calatayud@hotmail.com,748136760.0,...,Formación Profesional,Autónomo,Actualización de conocimientos,2814.00,5,Móvil,Noche,Híbrido,Sofía Fernández,53.0
3,8808cd20-a01b-41b1-8cbb-22fe18f1314b,2,Desarrollo Web Full Stack,2025-04-14,Email Marketing,Email_Promo,Telmo,Antón Andrés,telmo.anton@gmail.com,709572486.0,...,Grado Universitario,Desempleado,Actualización de conocimientos,5255.00,4,Ordenador,Fin de semana,Online,Carlos Martínez,77.0
4,f17f1eb6-babe-44c4-a08c-2be3d6498319,4,Ciberseguridad,2025-03-15,Referido,Referral_Alumnos,Isaías,Guerrero Vazquez,isaias.guerrero@hotmail.com,667403959.0,...,Grado Universitario,Desempleado,Actualización de conocimientos,3225.00,3,Móvil,Fin de semana,Presencial,Ana García,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105240,e07d5dce-456a-4718-ba3a-ffa1ccd64e55,6,Data Science,2024-11-06,FACEBOOK ADS,FB_Career_2025,Luz,Calzada Camacho,luz.calzada@gmail.com,NaN,...,Formación Profesional,AUTÓNOMO,EMPRENDER,0.00,2,Móvil,Tarde,PRESENCIAL,Carlos Martínez,NaN
105241,A42FDCBF-E951-4D38-B860-576F5FB7BA45,NaN,Data Science,2025-01-12,TIKTOK,TT_Challenge,Laura,Barberá Almagro,laura.barbera@gmail.com,NaN,...,Máster,EMPLEADO TIEMPO COMPLETO,Mejora salarial,12645.86,3,Móvil,Tarde,Presencial,Ana García,NaN
105242,Dc5a9362-b4b8-40f3-a7f3-6f91996de995,3,IA DEVELOPER,2024-10-24,Google ads,Gg_remarketing,Rafael,Espinosa Fuertes,rafael.espinosa@hotmail.com,NaN,...,Formación Profesional,EMPLEADO TIEMPO COMPLETO,COMPLEMENTAR FORMACIÓN ACTUAL,0.00,4,MÓVIL,Noche,Híbrido,Ana García,NaN
105243,Aaea5f50-707f-43b4-9278-eea79ff5b5f9,NaN,CIBERSEGURIDAD,2024-08-04,Google Ads,Gg_remarketing,Adelia,Pozuelo Blázquez,adelia.pozuelo@gmail.com,NaN,...,MÁSTER,EMPLEADO TIEMPO COMPLETO,Mejora salarial,2731.18,2,ORDENADOR,FLEXIBLE,Presencial,Miguel González,NaN


In [140]:
df_seguimiento['curso_final_id']

0         6
1         6
2         2
3         2
4         4
         ..
102297    5
102298    5
102299    1
102300    6
102301    1
Name: curso_final_id, Length: 102302, dtype: object

In [141]:
df_leads.to_csv('../data_clean/leads_final.csv', sep=';', index=False)
cursos_final.to_csv('../data_clean/cursos_final.csv', sep=';', index=False)
df_seguimiento.to_csv('../data_clean/seguimiento_final.csv', sep=';', index=False)
df_contacto.to_csv('../data_clean/primer_contacto_final.csv', sep=';', index=False)
df_tecnologias.to_csv('../data_clean/tecnologias_final.csv', sep=';', index=False)